In [1]:
import numpy as np
import pandas as pd
import sys

import config as cfg
from functools import reduce
import joblib
from tkinter import _flatten
sys.path.append("./tools/")
import funclib,os
import exact_ec_from_uniprot as exactec
import minitools as mtool
import benchmark_common as bcommon
import embedding_esm as esmebd
from tools.Attention import Attention
from keras.models import load_model
from pandarallel import pandarallel 
pandarallel.initialize() 
import benchmark_train as btrain

%load_ext autoreload
%autoreload 2

2023-02-06 15:27:52.297270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
input_fasta = cfg.DATADIR + 'sample_25.fasta'
# 1. 读入数据
print('step 1: loading data')
input_df = funclib.load_fasta_to_table(input_fasta) # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db

step 1: loading data


In [3]:
find_data=latest_sprot[latest_sprot.seq.isin(input_df.seq)]
noExist_data = input_df[~input_df.seq.isin(find_data.seq)]
exist_data = find_data.merge(input_df, on='seq', how='left').iloc[:,np.r_[8,0,1:8]].rename(columns={'id_x':'uniprot_id','id_y':'input_id'}).reset_index(drop=True)

In [4]:
# 3. EMBedding
mode='r'
print('step 3: Embedding')
if mode =='p':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=noExist_data, model='esm1b_t33_650M_UR50S',seqthres=1022)
if mode == 'r':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=input_df, model='esm1b_t33_650M_UR50S',seqthres=1022)

step 3: Embedding


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:40<00:00,  1.36s/it]


In [5]:
# 4. sequence alignment
print('step 4: sequence alignment')
if not os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)

step 4: sequence alignment


In [6]:
if mode =='p':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=noExist_data)
if mode == 'r':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)
    
    
blast_res =blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2,3:10]].rename(columns={'id_x':'input_id','id_y':'uniprot_id'}).reset_index(drop=True)


Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/uniprot_blast_db/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


In [8]:
pred_dmlf = pd.DataFrame(rep32.id.copy())
model_isEnzyme = load_model(cfg.ISENZYME_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_isEnzyme.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t1=joblib.load(cfg.DICT_LABEL_T1)
pred_dmlf['dmlf_isEnzyme']=(encoder_t1.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]

1/1 [==============================] - 0s 391ms/step


In [9]:
pred_dmlf

,id,dmlf_isEnzyme
0,sp|O00625|PIR_HUMAN,False
1,sp|P06028|GLPB_HUMAN,False
2,sp|P06746|DPOLB_HUMAN,False
3,sp|P09588|H2A_CAEEL,False
4,sp|P0A7M9|RL31_ECOLI,False
5,sp|P0DJD7|PEPA4_HUMAN,False
6,sp|P24941|CDK2_HUMAN,False
7,sp|P30843|BASR_ECOLI,False
8,sp|P49842|STK19_HUMAN,False
9,sp|P58107|EPIPL_HUMAN,False


In [10]:
model_howmany = load_model(cfg.HOWMANY_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_howmany.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t2=joblib.load(cfg.DICT_LABEL_T2)
pred_dmlf['dmlf_howmany']=(encoder_t2.inverse_transform(bcommon.props_to_onehot(predicted))).reshape(1,-1)[0]

1/1 [==============================] - 0s 386ms/step


In [11]:
model_ec = load_model(cfg.EC_MODEL,custom_objects={"Attention": Attention}, compile=False)
predicted = model_ec.predict(np.array(rep32.iloc[:,1:]).reshape(rep32.shape[0],1,-1))
encoder_t3=joblib.load(cfg.DICT_LABEL_T3)
pred_dmlf['dmlf_ec']=[','.join(item) for item in (encoder_t3.inverse_transform(bcommon.props_to_onehot(predicted)))]

1/1 [==============================] - 0s 380ms/step


In [12]:
pred_dmlf

,id,dmlf_isEnzyme,dmlf_howmany,dmlf_ec
0,sp|O00625|PIR_HUMAN,False,1,1.1.2.7
1,sp|P06028|GLPB_HUMAN,False,1,3.1.1.1
2,sp|P06746|DPOLB_HUMAN,False,1,2.4.2.4
3,sp|P09588|H2A_CAEEL,False,5,3.1.13.2
4,sp|P0A7M9|RL31_ECOLI,False,3,3.4.22.-
5,sp|P0DJD7|PEPA4_HUMAN,False,4,1.1.1.35
6,sp|P24941|CDK2_HUMAN,False,1,2.7.7.23
7,sp|P30843|BASR_ECOLI,False,1,3.1.27.3
8,sp|P49842|STK19_HUMAN,False,1,2.4.2.3
9,sp|P58107|EPIPL_HUMAN,False,3,2.4.2.3


In [13]:
results = pred_dmlf.merge(exist_data, left_on='id',right_on='input_id', how='left') 

In [14]:
results=results.fillna('#')
results['ec_pred'] =results.apply(lambda x : x.ec_number if x.ec_number!='#' else ('-' if x.dmlf==False else x.dmlf_ec) ,axis=1)
results = results[['id', 'ec_pred']].rename(columns={'id':'id_input'})

In [15]:
results

,id_input,ec_pred
0,sp|O00625|PIR_HUMAN,1.13.11.24
1,sp|P06028|GLPB_HUMAN,-
2,sp|P06746|DPOLB_HUMAN,"4.2.99.-,2.7.7.7"
3,sp|P09588|H2A_CAEEL,-
4,sp|P0A7M9|RL31_ECOLI,-
5,sp|P0DJD7|PEPA4_HUMAN,3.4.23.1
6,sp|P24941|CDK2_HUMAN,2.7.11.22
7,sp|P30843|BASR_ECOLI,-
8,sp|P49842|STK19_HUMAN,2.7.11.1
9,sp|P58107|EPIPL_HUMAN,-
